In [ ]:
# # Code Splitter by LLama Index
from llama_index.core.node_parser import CodeSplitter
from llama_index.readers.file import FlatReader
from pathlib import Path


documents = []

for code_file in code_files:
    raw_documents = FlatReader().load_data(Path(code_file))
    splitter = CodeSplitter(
        language="python",
        chunk_lines=3,  # lines per chunk
        chunk_lines_overlap=1,  # lines overlap between chunks
        max_chars=1500,  # max chars per chunk
    )
    nodes = splitter.get_nodes_from_documents(raw_documents)

    for node in nodes:
        print(node.text)
        print("=======================================")
        document = Document(
            page_content=node.text,
            metadata={
                "filename": node.metadata['filename'],
            },
        )
        documents.append(document)

In [ ]:
# Using OllamaLLM

from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM


B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS»>\n", "\n<</SYS>>\n\n"

system_prompt = """You are a helpful assistant, you will use the provided context to answer user questions.
Read the given context before answering questions and think step by step. If you can not answer a user question based on the provided context, inform the user. Do not use any other information for answering user"""
    
instruction = """
Context: {context}
User: {question}"""

def prompt_format(instruction=instruction, system_prompt=system_prompt):
    SYSTEM_PROMPT = B_SYS + system_prompt + E_SYS
    prompt_template = B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

template = prompt_format()

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model="codellama:13b", callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]), temperature=0)

memory = ConversationSummaryMemory(
    llm=chat_model, memory_key="chat_history", return_messages=True
)

qa = ConversationalRetrievalChain.from_llm(
    model,
    retriever=retriever
)

result = qa({'question': "how do I add user?", 'chat_history': []})
# result['answer']


In [ ]:
# Using Commiunity Ollama

from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_community.llms import Ollama
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationSummaryMemory


chat_model = Ollama(
                base_url="http://localhost:11434",
                model="codellama:13b",
                callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
            )

memory = ConversationSummaryMemory(
    llm=chat_model, memory_key="chat_history", return_messages=True
)

qa = ConversationalRetrievalChain.from_llm(
    chat_model,
    retriever=retriever
)

result = qa({'question': "how do I add user?", 'chat_history': []})
print(result['answer'])